In [1]:
import cv2
import os
import tensorflow as tf
import numpy as np

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_virtual_device_configuration(
        gpus[0],[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5120)])
    except RuntimeError as e:
        print(e)

In [3]:
IMAGE_HEIGHT, IMAGE_WIDTH = 224, 224
SEQUENCE_LENGTH = 50

DATASET_DIR = 'train'

CLASSES_LIST = ["0", "1" ,"2", "3"]

In [4]:
def frames_extraction(video_path):
    frames_list = []
    video_reader = cv2.VideoCapture(video_path)
    video_frames_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))
    
    skip_frames_window = max(int(video_frames_count / SEQUENCE_LENGTH), 1)
    
    for frame_counter in range(SEQUENCE_LENGTH):
        video_reader.set(cv2.CAP_PROP_POS_FRAMES, frame_counter*skip_frames_window)
        
        success, frame = video_reader.read()
        
        if not success:
            break
            
        resized_frame = cv2.resize(frame, (IMAGE_HEIGHT, IMAGE_WIDTH))
        normalized_frame = resized_frame / 255
        
        frames_list.append(normalized_frame)
    video_reader.release()
    
    return frames_list

In [5]:
def create_dataset():
    features = []
    labels = []
    video_files_paths = []
    
    for class_index, class_name in enumerate(CLASSES_LIST):
        print(f'Extracting Data of Class : {class_name}')
        files_list = os.listdir(os.path.join(DATASET_DIR, class_name))
        
        for file_name in files_list:
            video_file_path = os.path.join(DATASET_DIR, class_name, file_name)
            frames = frames_extraction(video_file_path)
            if len(frames) == SEQUENCE_LENGTH:
                features.append(frames)
                labels.append(class_index)
                video_files_paths.append(video_file_path)
                
    features = np.asarray(features)
    labels = np.array(labels)
    
    return features, labels, video_files_paths

In [6]:
features, labels, video_files_paths = create_dataset()

Extracting Data of Class : 0
Extracting Data of Class : 1
Extracting Data of Class : 2
Extracting Data of Class : 3


MemoryError: Unable to allocate 18.7 GiB for an array with shape (334, 50, 224, 224, 3) and data type float64